In [1]:
# Импорт библиотек
import os
import cv2
import math
import numpy as np
from imutils import perspective

# детектор обнаружения маркера Aruco
parameters = cv2.aruco.DetectorParameters_create()
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)

# загрузка изображения и изменение размера 
img = cv2.imread("222v1.jpg")
scale = 1.6
wP = int(1006 // scale)
hP = int(1603 // scale)
pd = 0
img = img[pd:img.shape[0] - pd, pd:img.shape[1] - pd]

# Извлечение координат углов и индентификатора маркера Aruco
corners, ids, _ = cv2.aruco.detectMarkers(img, aruco_dict, parameters=parameters)

# Координаты внешних углов каждого маркера Aruco 
bg = np.array([[corners[2][0][0][0], corners[2][0][0][1]],
               [corners[3][0][0][0], corners[3][0][0][1]],
               [corners[1][0][1][0], corners[1][0][1][1]],
               [corners[0][0][3][0], corners[0][0][3][1]]])  
bg = perspective.order_points(bg) # сортируем координаты по часовой стрелке 
bg = np.int0(bg)

In [2]:
#isWritten = cv2.imwrite('image1.png', img)
cv2.imshow("Image", img)
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [4]:
# Преобразование индентификаторов и координат углов
ids = ids.flatten()

# Внешний многоугольник по внешним точкам каждого маркера
cv2.polylines(img, [bg], True, (0, 255, 0), 1)

# Индитификатор маркера 
for i,_ in enumerate(corners):
    # Координаты центра каждого маркера
    cY = int((corners[i][0][0][1] + corners[i][0][2][1]) / 2)
    cX = int((corners[i][0][0][0] + corners[i][0][2][0]) / 2) 
    cv2.circle(img, (cX, cY), 2, (0, 0, 255), -1)
    # Индентификатор маркера
    cv2.putText(img, str(ids[i]), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#isWritten = cv2.imwrite('image01.png', img)
cv2.imshow("Image", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [5]:
#перспективное преобразование, P и H фактические стороны внешнего прямоугольника в мм
def warpImg(img, bg, P=1603, H=1006):
    box = perspective.order_points(bg) # сортируем координаты по часовой стрелке          
    pts1 = np.float32(box) #точки для исходного изображения
    pts2 = np.float32([[0, 0], [P, 0], [P, H], [0, H]]) #точки для преобразования,P-ширина, H - высота
    matrix = cv2.getPerspectiveTransform(pts1, pts2) #  перспективное преобразование
    imgWarp = cv2.warpPerspective(img, matrix, (P, H)) # применить перспективное преобразование на изображение
    #imgWarp = cv2.resize(imgWarp, None, fx = 0.7, fy = 0.7, interpolation=cv2.INTER_NEAREST)
    return imgWarp

In [6]:
#Вывод картинки после преобразования
imgWarp = warpImg(img, bg, P= hP, H= wP)
#isWritten = cv2.imwrite('imgWarp024.png', imgWarp)
cv2.imshow("Image", imgWarp)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [7]:
# Масштабный коэффициент 
# Периметр внешнего многоугольника по внешним точкам каждого маркера
aruco_perimeter = cv2.arcLength(bg, True)
# Периметр рамки изображения imgWarp
pp = imgWarp.shape[1]*2 + imgWarp.shape[0]*2
# Коэффициент корректировки преобразования
rect2 = cv2.minAreaRect(bg)
(x2, y2), (w2, h2), angle = rect2
fg = pp/aruco_perimeter

# Фактическое значение периметра в мм внешнего многоугольника по внешним точкам каждого маркера
p = 5218

# Соотношение пикселей к мм
pk = aruco_perimeter * fg / p
pk

0.6243771559984669

In [8]:
#определение контуров
def getContours(img, thresh1=100,thresh2=50,thresh3=4000):
       
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #преобразов в градации серого
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 5) #размытие по Гауссу с ядром 3х3
    imgCanny = cv2.Canny(imgBlur, thresh1, thresh2 ) #обнаруживаем края

    #операция закрытие, чтобы получить четкие края
    imgDial = cv2.dilate(imgCanny, None, iterations=1) #расширяет края
    imgThre = cv2.erode(imgDial, None, iterations=1) #размывает края
   
   #RETR_EXTERNAL - внешние контуры, CHAIN_APPROX_SIMPLE - соединяет горизонтальные, вертикальные и диагональные контуры.
    cnts = cv2.findContours(imgThre, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # поиск внешних контуров
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    Countours = []
    for i in cnts:
        area = cv2.contourArea(i) #площадь объекта
        if area > thresh3:
            pr = cv2.arcLength(i, True) #периметр, True - замкнутый контур
            # приближение формы контура к другой форме с меньшим кол-вом вершин
            approx = cv2.approxPolyDP(i, 0.02*pr, True) #апроксимация
            bbox = cv2.boundingRect(approx) # Прямой ограничивающий прямоугольник
            Countours.append([area, approx, i]) 
                    
    #сортировка контуров в порядке убывания по размерам
    finalCountours = sorted(Countours, key=lambda x: x[0], reverse = True)
    
    return finalCountours

In [9]:
# Выбор параметров для обнаружения точек контуров 

def nothing(x):
    pass

cv2.namedWindow("track")
cv2.createTrackbar("T1", "track", 1, 255, nothing)
cv2.createTrackbar("T2", "track", 1, 255, nothing)
cv2.createTrackbar("T3", "track", 1, 10000, nothing)

while True:
    
    k = cv2.getTrackbarPos("T1", "track")
    m = cv2.getTrackbarPos("T2", "track") 
    s = cv2.getTrackbarPos("T3", "track")
    
    image1 = imgWarp.copy()
    cnt = getContours(image1, thresh1 = k, thresh2 = m, thresh3 = s)
    for i, elem in enumerate(cnt):
        cv2.drawContours(image1, cnt[i][1], -1, (0, 255, 0), 4)
        cv2.imshow("Image", image1)
           
    k = cv2.waitKey(1)& 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()

In [11]:
# Функция для расчета фокусного расстояния
m_distance = 2067 # расстояние в мм от камеры до плоскости с маркерами
r_width = 1006 # ширина в мм внешнего прямоугольника 
# ширина в пикселях внешнего прямоугольника 
width_in_image = imgWarp.shape[0]
# функция определения фокусного расстояния 
def FL(m_distance, r_width, width_in_image): 
    foc_length = (width_in_image * m_distance)/ r_width 
    return foc_length 

FL = FL(m_distance, r_width, width_in_image)

In [14]:
# Точки данных для полиномиальной регрессии
#периметр объекта в пикселях 
newlist = [650,648,556,480,432,372,348,351,320,294,305,304,301,
 278,277,269,267,322]
#расстояние от камеры до маркера DICT_4X4_100 Aruco
y3 = [367, 470, 561, 665, 765, 875, 966, 1065, 1160, 
      1264, 1370, 1472, 1569, 1673, 1777, 1877, 1977, 2067]
coff1 = np.polyfit(newlist, y3, 2)  # зависимость y = Ax^2 + Bx + C


In [15]:
# определение размеров 
def magic(imgWarp, pk, m_distance, coff1):
    
    # детектор обнаружения маркера DICT_4X4_100 Aruco
    parameters = cv2.aruco.DetectorParameters_create()
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_100)
    # Извлечение координат углов и индентификатора маркера Aruco
    corners, ids, _ = cv2.aruco.detectMarkers(imgWarp, aruco_dict, parameters=parameters)
    
    cnts = getContours(imgWarp) #получение контуров на изображении 
    
    if ids:
        # Координаты углов маркера Aruco 
        sg = np.array([[corners[0][0][0][0], corners[0][0][0][1]],
                       [corners[0][0][1][0], corners[0][0][1][1]],
                       [corners[0][0][2][0], corners[0][0][2][1]],
                       [corners[0][0][3][0], corners[0][0][3][1]]])  
        sg = perspective.order_points(sg) # сортируем координаты по часовой стрелке 
        sg = np.int0(sg)

        # Периметр внешнего многоугольника по внешним точкам каждого маркера
        ar_per1 = int(cv2.arcLength(sg, True))
    
        rect2 = cv2.minAreaRect(sg)
        (x2, y2), (w2, h2), angle = rect2

        pr = 100*4 # периметр в мм 
        # Соотношение пикселей к мм
        k = ar_per1 / pr
        
        # расстояние от камеры до объекта в мм
        A, B, C = coff1
        distanceMM = A * ar_per1**2 + B * ar_per1 + C
        # высота объекта
        height = m_distance - distanceMM
        # Определение пересечения контрура маркера DICT_4X4_100 Aruco с контуром объекта
        for i, elem in enumerate(cnts):
            for j, el in enumerate(sg):
                pt = tuple([int(round(sg[j][0])), int(round(sg[j][1]))])
                #проверка на пересечение контуров маркера и объектов
                dist = cv2.pointPolygonTest(cnts[i][2], pt, False)
                if dist > 0:
                    # рамка ограничивающая контуры объектов 
                    rect = cv2.minAreaRect(cnts[i][1])
                    (x, y), (w, h), angle = rect   
                    
                    # Ширина и длина объектов, мм
                    ob_width = w // k
                    ob_length = h // k
                    # Прямоугольник
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)

                    cv2.circle(imgWarp, sg[0], 2, (0, 255, 0), -1)
                    cv2.polylines(imgWarp, [box], True, (0, 255, 0), 1)
                    if ob_width <= ob_length:
                        wk = ob_width
                        wd = ob_length
                    else:
                        wk = ob_length
                        wd = ob_width
                    cv2.putText(imgWarp, "width {} mm".format(round(wk, 1)), (int(x +80), 
                                int(y -10)), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
                    cv2.putText(imgWarp, "length {} mm".format(round(wd, 1)), (int(x +80), 
                                int(y+15)), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
                    cv2.putText(imgWarp, "height {} mm".format(round(height, 1)), (int(x +80), 
                                int(y + 40)), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

                else:
                    # Ширина и длина объекта, мм
                    ob_w = w2 // k
                    ob_l = h2 // k
                    cv2.circle(imgWarp, sg[0], 2, (255, 200, 0), -1)
                    cv2.polylines(imgWarp, [sg], True, (255, 200, 0), 1)
                    if ob_w <= ob_l:
                        wk = ob_w
                        wd = ob_l
                    else:
                        wk = ob_l
                        wd = ob_w
                    cv2.putText(imgWarp, "width {} mm".format(round(wk, 1)), (int(x2 -100), 
                                int(y2 - 10)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 200, 0), 2)
                    cv2.putText(imgWarp, "length {} mm".format(round(wd, 1)), (int(x2 - 100), 
                                int(y2 + 15)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 200, 0), 2)
                    cv2.putText(imgWarp, "height {} mm".format(round(height, 1)), (int(x2 - 100), 
                                int(y2 + 40)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 200, 0), 2)
    else:
        for i, elem in enumerate(cnts):
            # рамка ограничивающая контур 
            rect = cv2.minAreaRect(cnts[i][1])
            (x, y), (w, h), angle = rect 

            # Ширина и длина объектов, мм
            ob_width = w // pk
            ob_length = h // pk
           
            # Прямоугольник
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            cv2.circle(imgWarp, (int(x), int(y)), 3, (255, 255, 0), -1)
            cv2.polylines(imgWarp, [box], True, (255, 255, 0), 1)

            if ob_width <= ob_length:
                wk = ob_width
                wd = ob_length
            else:
                wk = ob_length
                wd = ob_width

            cv2.putText(imgWarp, "width {} mm".format(round(wk, 1)), (int(x -100), 
                        int(y - 10)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 2)
            cv2.putText(imgWarp, "length {} mm".format(round(wd, 1)), (int(x - 100), 
                        int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 2)
           
    return imgWarp

In [16]:
img2 = imgWarp.copy()
img3 = magic(img2, pk, m_distance, coff1)
#isWritten = cv2.imwrite('imgWarp22.png', imgWarp)
cv2.imshow("Image", img3)
cv2.waitKey(6000)
cv2.destroyAllWindows()

In [17]:
# Load Cap
webcam = True
path = '222v1.jpg'
cap = cv2.VideoCapture("222v.mp4")
#cap.set(10, 150)
# Если не удалось открыть файл, выводим сообщение об ошибке
if cap.isOpened() == False:
    print('Не возможно открыть файл')
    
while True:
    
    if webcam:
        success, img = cap.read()
        # если кадры закончились, то выход
        if img is None:
            break
    else:
        img = cv2.imread(path) #считываем данные графического файла в переменную
        pd = 0
        img = img[pd:img.shape[0] - pd, pd:img.shape[1] - pd]
    
    imgW = warpImg(img, bg, P= hP, H= wP)    
    img = magic(imgW, pk, m_distance, coff1)
    
    cv2.imshow('Original', img)
        
    if cv2.waitKey(10)& 0xFF == ord("q"):
        break
        
cap.release()

cv2.destroyAllWindows()